In [0]:
%sql
create table if not exists silver.events
using delta
as select * from delta.`/Volumes/workspace/ecommerce/delta/silver/events`;

In [0]:
spark.sql("SELECT * FROM silver.events WHERE event_type='view'").explain(True)

In [0]:
# Partitioned table
spark.sql("""
  CREATE TABLE silver.events_part
  USING DELTA
  PARTITIONED BY (event_week, event_type)
  AS SELECT
    *,
    WEEKOFYEAR(event_date) as event_week
FROM silver.events
""")


In [0]:
%sql
SHOW PARTITIONS silver.events_part

In [0]:
%sql
OPTIMIZE silver.events_part ZORDER BY (user_id, product_id);


In [0]:
%sql
SELECT user_id
FROM silver.events
ORDER BY rand()
LIMIT 1;


In [0]:
%sql
DESCRIBE HISTORY silver.events_part

In [0]:
# Benchmark
import time
start = time.time()
spark.sql("SELECT * FROM silver.events WHERE user_id=509341569").count()
print(f"Time: {time.time()-start:.2f}s")

In [0]:
# Cache for iterative queries
cached = spark.table("silver.events")
cached.count()  # Materialize

In [0]:
cached = spark.table("silver.events_part")
cached.count()  # Materialize